In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import os

## Data Loading and Preparation¶

### Loading Data¶

Cek every database

In [ ]:
files=os.listdir(r'../input/uber-pickups-in-new-york-city')[-7:]
files

In [ ]:
#uber-raw-data-jul14
final = pd.read_csv('../input/uber-pickups-in-new-york-city/uber-raw-data-jul14.csv')

In [ ]:
final.shape

###  Data Preparation¶

##### Lat : The latitude of the Uber pickup

##### Lon : The longitude of the Uber pickup

##### Base : The TLC base company code affiliated with the Uber pickup

##### The globe is split into an imaginary 360 sections from both top to bottom (north to south) and 180 sections from side to side (west to east). The sections running from top to bottom on a globe are called longitude, and the sections running from side to side on a globe are called latitude.


##### Latitude is the measurement of distance north or south of the Equator.

##### Every location on earth has a global address. Because the address is in numbers, people can communicate about location no matter what language they might speak. A global address is given as two numbers called coordinates. The two numbers are a location's latitude number and its longitude number ("Lat/Long").

In [ ]:
df=final.copy()

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.shape

In [ ]:
df.isna().sum()

In [ ]:
df.dtypes

In [ ]:

df['Date/Time'] = pd.to_datetime(df['Date/Time'], format="%m/%d/%Y %H:%M:%S")

In [ ]:
df.dtypes

In [ ]:

df['weekday']=df['Date/Time'].dt.day_name()
df['day']=df['Date/Time'].dt.day
df['minute']=df['Date/Time'].dt.minute
df['month']=df['Date/Time'].dt.month
df['hour']=df['Date/Time'].dt.hour

In [ ]:
df.dtypes

In [ ]:
df.head()

In [ ]:
df['Base'].unique()

In [ ]:
df['day'].unique()

In [ ]:
df['weekday'].unique()

#### Analysis of journey by Week-days

In [ ]:
import plotly.express as px

In [ ]:
px.bar(x=df['weekday'].value_counts().index,
      y=df['weekday'].value_counts().values
      )

#### seems to have highest sales on Thursday

### Analysis by Hour

In [ ]:
plt.hist(df['hour'])

#### It peaks during evening time when people are logging off from work

In [ ]:
for i,month in enumerate(df['month'].unique()):
    print(month)

In [ ]:

plt.figure(figsize=(40,20))
for i,month in enumerate(df['month'].unique()):
    plt.subplot(3,2,i+1)
    df[df['month']==month]['hour'].hist()


#### Analysis of Rush of each hour in each month

In [ ]:
for i in df['month'].unique():
    plt.figure(figsize=(5,3))
    df[df['month']==i]['hour'].hist()
    

#### analysis of which month has max rides

In [ ]:
pip install chart_studio

In [ ]:
# !pip install chart_studio
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [ ]:
trace1 = go.Bar( 
        x = df.groupby('month')['hour'].sum().index,
        y = df.groupby('month')['hour'].sum(),
        name= 'Priority')
iplot([trace1])

#### Analysis of Journey of Each  Day

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(df['day'], bins=30, rwidth=.8, range=(0.5, 30.5))
plt.xlabel('date of the month')
plt.ylabel('Total Journeys')
plt.title('Journeys by Month Day')

#### Analysis of Total rides month wise

In [ ]:
plt.figure(figsize=(20,8))
for i,month in enumerate(df['month'].unique(),1):
    plt.subplot(3,2,i)
    df_out=df[df['month']==month]
    plt.hist(df_out['day'])
    plt.xlabel('days in month'.format(i))
    plt.ylabel('total rides')

### getting Rush in hour 

In [ ]:
lat_df = df[["Lat","Lon"]] 

In [ ]:
df["hour"].isna().sum()

In [ ]:
plt.figure(figsize=(20,8))
sns.set_style(style='whitegrid')
sns.pointplot(x="hour",y="Lat",data=df)

##### adding hue params

In [ ]:
sns.set(rc={'figure.figsize':(11.7,8.27)})
ax=sns.pointplot(x="hour",y="Lat", hue="weekday",data=df)
ax.set_title('hoursoffday vs latiitide of passenger')


#### to analyse which base number gets popular by month name

In [ ]:
df.head()

In [ ]:
df['Base'].head()

In [ ]:
df.groupby(['Base','month'])['Date/Time'].count()

In [ ]:
base=df.groupby(['Base','month'])['Date/Time'].count().reset_index()
base

#### 2 Cross Analysis
#### Through our exploration we are going to visualize:


#### 1.Heatmap by Hour and Weekday.
#### 2.Heatmap by Hour and Day.
#### 3.Heatmap by Month and Day.
#### 4.Heatmap by Month and Weekday.

## Heatmap by Hour and Weekday.

#### create pivot_tables

##### simplest way of creating pivot tables,first of all call groupby on 2 columns so that we will get groups 
##### df.groupby(['weekday','hour']).apply(lambda x: len(x)), now "weekday" becomes rows and "hour" becomes cols
##### & then call unstack

In [ ]:
def count_rows(rows):
    return len(rows)

In [ ]:
by_cross = df.groupby(['weekday','hour']).apply(count_rows)
by_cross

In [ ]:
pivot=by_cross.unstack()
pivot

##### creating heatmap so that it can be easily visualize

In [ ]:
plt.figure(figsize=(10,6))
sns.heatmap(pivot, annot=False)

In [ ]:
df.head()

In [ ]:
def heatmap(col1,col2):
    by_cross = df.groupby([col1,col2]).apply(lambda x:len(x))
    pivot=by_cross.unstack()
    plt.figure(figsize=(10,6))
    return sns.heatmap(pivot,annot=False)

In [ ]:
## validating above Analysis through Heatmap
heatmap('day','hour')

In [ ]:
heatmap('day','month')

#### Analysis of Location data points¶

In [ ]:
plt.figure(figsize=(10,6))

plt.plot(df['Lon'], df['Lat'],'r+', ms=0.5)
plt.xlim(-74.2, -73.7)
plt.ylim(40.6,41)

### perform Spatial Analysis using heatmap to get a clear cut of Rush on Sunday(Weekend)

In [ ]:
df.head()

In [ ]:
df_out=df[df['weekday']=='Sunday']
df_out.head()

In [ ]:
df_out.groupby(['Lat','Lon'])['weekday'].count().reset_index()

In [ ]:
from folium.plugins import HeatMap

In [ ]:
import folium
from folium.plugins import HeatMap
basemap=folium.Map()

In [ ]:
HeatMap(df_out.groupby(['Lat','Lon'])['weekday'].count().reset_index(),zoom=20,radius=15).add_to(basemap)
basemap

##### Lets create a function for a specific day

In [ ]:
def plot(df,day):
    df_out=df[df['weekday']==day]
    df_out.groupby(['Lat','Lon'])['weekday'].count().reset_index()
    HeatMap(df_out.groupby(['Lat','Lon'])['weekday'].count().reset_index(),zoom=20,radius=15).add_to(basemap)
    return basemap


In [ ]:
plot(df,'Sunday')

## Analysis of Jan-June uber_15

In [ ]:
uber_15 = pd.read_csv(r'../input/uber-pickups-in-new-york-city/uber-raw-data-janjune-15.csv',encoding='utf-8')
uber_15.head()

In [ ]:
uber_15.shape

In [ ]:
#Checking the minimum date in the uber_15
uber_15['Pickup_date'].min()

In [ ]:
#Checking the maximum date in the uber_15
uber_15['Pickup_date'].max()

In [ ]:
uber_15['Pickup_date'] =  pd.to_datetime(uber_15['Pickup_date'], format='%Y-%m-%d %H:%M:%S')

In [ ]:

uber_15['weekday']=uber_15['Pickup_date'].dt.day_name()
uber_15['day']=uber_15['Pickup_date'].dt.day
uber_15['minute']=uber_15['Pickup_date'].dt.minute
uber_15['month']=uber_15['Pickup_date'].dt.month
uber_15['hour']=uber_15['Pickup_date'].dt.hour

In [ ]:
uber_15.head()

##### Uber pickups by the month in NYC

In [ ]:
px.bar(x=uber_15['month'].value_counts().index,
           y=uber_15['month'].value_counts().values)

#### We can see that the number of Uber pickup has been steadily increasing throughout the first half of 2015 in NYC

#### Analysing Rush in New york City

In [ ]:
ax=sns.countplot(uber_15['hour'])
ax

##### Interestingly, after the morning rush, the number of Uber pickups doesn't dip much throughout the rest of the morning and early afternoon. There is significantly more demand in the evening than the daytime. Let's investigate to see if there's a difference in hourly pattern for different days of the week.

### Analysing In-Depth Analysis of Rush in New york City Day & hour wise

##### group the data by Weekday and hour

In [ ]:
uber_15.groupby(['weekday', 'hour'])['Pickup_date'].count()

In [ ]:
uber_15.groupby(['weekday', 'hour'])['Pickup_date'].count().reset_index()

In [ ]:
summary=uber_15.groupby(['weekday', 'hour'])['Pickup_date'].count().reset_index()

In [ ]:
summary=summary.rename(columns = {'Pickup_date':'Counts'})
summary

In [ ]:
plt.figure(figsize=(10,6))
sns.pointplot(x="hour", y="Counts", hue="weekday", data=summary)

##### Loading Uber-Jan-Feb-FOIL.csv

In [ ]:
uber_foil=pd.read_csv(r'../input/uber-pickups-in-new-york-city/Uber-Jan-Feb-FOIL.csv')

In [ ]:
uber_foil.head()

In [ ]:
uber_foil['dispatching_base_number'].unique()

In [ ]:
sns.boxplot(x = 'dispatching_base_number', y = 'active_vehicles', data = uber_foil)

#### seems to have more number of Active Vehicles in B02764

In [ ]:
sns.boxplot(x = 'dispatching_base_number', y = 'trips', data = uber_foil)

#### seems to have more number of trips in B02764

In [ ]:
# Finding the ratio of trips/active_vehicles
uber_foil['trips/vehicle'] = uber_foil['trips']/uber_foil['active_vehicles']

In [ ]:
uber_foil.head()

In [ ]:
uber_foil.set_index('date')

##### how Average trips/vehicle inc/decreases with dates with each of base umber

In [ ]:
plt.figure(figsize=(10,6))
uber_foil.set_index('date').groupby(['dispatching_base_number'])['trips/vehicle'].plot()
plt.ylabel('Average trips/vehicle')
plt.title('Demand vs Supply chart (Date-wise)')
plt.legend()